In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb


DirPath = "./tmp_GenerateResources/"
print("ベンチマーククラスを固定した関数ごとのMAPE表を作成")
for benchmark in benchmarks:
    print(f"benchmark={benchmark}")
    try:
        ModelDFfixedClass = return_FixedClassModelDF(benchmark=benchmark)
    except:
        print(f"return_FixedClassModelDF({benchmark})の実行に失敗しました")
        continue
    ModelDFfixedClass_LowestMape = ModelDFfixedClass
    index = ModelDFfixedClass_LowestMape.index.tolist()
    columns = ModelDFfixedClass_LowestMape.columns.tolist()
    for column in columns:
        for row in index:
            ModelDFfixedClass_LowestMape.at[row, column] = ModelDFfixedClass.at[row, column].mape_score_InTrain
    average = ModelDFfixedClass_LowestMape.mean()
    average.name = "Average"
    ModelDFfixedClass_LowestMape.append(average)
    ModelDFfixedClass_LowestMape.to_csv(f"{DirPath}{benchmark}FixedClassMapeTable.csv")
DirPath = "./tmp_GenerateResources/"
print("実行プロセス数を固定した関数ごとのMAPE表を作成")
for benchmark in benchmarks:
    try:
        ModelDFfixedProcess = return_FixedProcessModelDF(benchmark=benchmark)
    except:
        print(f"return_FixedProcessModelDF({benchmark})の実行に失敗しました")
        continue
    ModelDFfixedProcess_LowestMape = ModelDFfixedProcess
    index = ModelDFfixedProcess_LowestMape.index.tolist()
    columns = ModelDFfixedProcess_LowestMape.columns.tolist()
    for column in columns:
        for row in index:
            ModelDFfixedProcess_LowestMape.at[row, column] = ModelDFfixedProcess.at[row, column].mape_score_InTrain
    average = ModelDFfixedProcess_LowestMape.mean()
    average.name = "Average"
    ModelDFfixedProcess_LowestMape.to_csv(f"{DirPath}{benchmark}FixedProcessMapeTable.csv")

In [ ]:
ModelDFfixedProcess_LowestMape